In [4]:
import datetime
import os
from itertools import combinations
import copy
import multiprocessing
import math
from collections import deque
import noise

#This is the array of Graph6 outputs
outputLineQueue = multiprocessing.Queue()

testNumber = 0

imgDirectory = "multi grid test"


def bfs_2d_array(grid):
    # Define the directions: up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Get the dimensions of the grid
    rows = len(grid)
    cols = len(grid[0])

    # Create a queue for BFS
    queue = deque()
    queue.append([(0, 0)])  # Start with the top left corner

    # Store the paths to reach the goal
    paths = []

    while queue:
        path = queue.popleft()
        row, col = path[-1]

        # Check if we have reached a 2
        if grid[row][col] == 2:
            paths.append(path)
        
        # Explore the neighboring cells
        for dr, dc in directions:
            new_row = row + dr
            new_col = col + dc

            # Check if the new cell is within the grid boundaries
            if 0 <= new_row < rows and 0 <= new_col < cols:

                # Check if the new cell is a traversable cell (1)
                if grid[new_row][new_col] == 1:

                    # Check if the new cell has already been visited in the current path
                    if (new_row, new_col) not in path:

                        # Add the new cell to the current path
                        new_path = path + [(new_row, new_col)]
                        queue.append(new_path)

    return paths

def display_grid(grid):
    # Create a colormap for the grid cells
    cmap = plt.cm.colors.ListedColormap(['black', 'white', 'red', 'green'])
    bounds = [0, 0.5, 1.5, 2.5, 3]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

    # Create a figure and axis
    fig, ax = plt.subplots()

    # Create a color mesh for the grid cells
    ax.imshow(grid, cmap=cmap, norm=norm)

    # Set ticks and labels for the grid cells
    ax.set_xticks(np.arange(len(grid[0])))
    ax.set_yticks(np.arange(len(grid)))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(length=0)

    # Add gridlines
    ax.grid(color='black', linestyle='-', linewidth=1)

    # Loop through all the cells and add text annotations
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] != 0:
                ax.text(j, i, str(grid[i][j]), ha='center', va='center')

    # Show the plot
    plt.show()


def generate_grids(size, num_grids):
    grids = []

    for _ in range(num_grids):
        grid = [[0 for _ in range(size)] for _ in range(size)]

        # Generate Perlin noise for cell placement
        for i in range(size):
            for j in range(size):
                value = noise.pnoise2(i / size, j / size)
                if value > 0:
                    grid[i][j] = 1

        # Change three random cells to 2
        indices = [(i, j) for i in range(size) for j in range(size)]
        random.shuffle(indices)
        for k in range(3):
            i, j = indices[k]
            grid[i][j] = 2

        grids.append(grid)

    return grids
    
    
def multiGrid(gridArrays):
    
    print(len(gridArrays))
    
    #for grid in gridArrays:
        #bfs_2d_array(grid)


def multithreading2(min_size, max_size, gridNum, num_threads):
    
    global imgDirectory
    global currentGraphFile
    global outputStream
    
    if (not os.path.exists(imgDirectory)):
        os.makedirs(imgDirectory)
    
    # Create a thread pool executor with the specified number of threads

    print("Multithread 2 enabled")
    
    
    
    #Make a 2D array for thread tasks (will become 3D later in code)
    #1st dimension stores the thread to use
    #2nd dimension stores the graph arguments that each thread is in charge of
    #3rd dimension stores the arguments for each graph
    threadTasks = []
    for i in range(num_threads):
        threadTasks.append([])
        
    taskAssign = 0
    
    pool = multiprocessing.Pool()
    
    # Submit each number to the executor for processing

    #futures = []

    
    
    allGraphs = 0

    for num_nodes in range(min_size, max_size+1):
        
        grids = generate_grids(num_nodes, gridNum)
        

        for g in grids:
            
            display_grid(g)
            
            threadTasks[taskAssign].append(g)

            taskAssign = taskAssign + 1
            allGraphs = allGraphs + 1

            if (taskAssign == num_threads):
                taskAssign = 0



    print(f"{allGraphs} graphs searched")
    
    # This actually does the real multithreading
    pool.map(multiGrid, threadTasks)
    
    #This waits for the threads to finish
    pool.close()
    pool.join()
        
        #results = [future.result() for future in futures]
        
    saveFiles()

    return


min_nodes = int(input("Enter the min grid size: "))
max_nodes = int(input("Enter the max grid size: "))

gridNum = int(input("Enter the number of grids for each size: "))

threads = int(input("Enter the number of threads: "))


print("Starting!")

start_time = datetime.datetime.now()


multithreading2(min_nodes, max_nodes, gridNum, threads)


end_time = datetime.datetime.now()

elapsed_time = end_time - start_time
totalSeconds = elapsed_time.total_seconds()



hours, remainder = divmod(int(totalSeconds), 3600)
minutes, seconds = divmod(remainder, 60)
roundSeconds = float("{:.2f}".format(math.modf(totalSeconds)[0] + seconds))

print("Elapsed time: {} hours, {} minutes, {} seconds".format(int(hours), int(minutes), roundSeconds))

ModuleNotFoundError: No module named 'noise'